In [301]:
import pandas as pd 
import csv


In [302]:
with open ('file_list.txt', 'r') as f:
    csv_list= f.readlines()
file_dict = {}
for csv in csv_list:
    key='ZippedData/'+csv.strip()
    df_name=csv.strip()
    df_name=df_name.split('.csv.gz')[0]
    df=pd.read_csv(key, compression='gzip')
    file_dict[df_name] = df
file_dict.keys

<function dict.keys>

In [303]:
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )

rt_movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )



In [304]:
rt_reviews_info = rt_movie_info[['id', 'director', 'runtime', 'rating']]

rt_reviews_cleaned = rt_reviews[['id', 'rating', 'critic']]
rt_df = rt_reviews_info.merge(rt_reviews_cleaned, how = 'inner', on = 'id')

rt_df.head(1)

,id,director,runtime,rating_x,rating_y,critic
0,3,David Cronenberg,108 minutes,R,3/5,PJ Nabarro


In [305]:
# NORMALIZE RATINGS RATING>FLOAT

def norm_ratings(rating):
    grades = {"A":10,"B":8,"C":6,"D":4,"F":2,"A+":10,"B+":9,"C+":7,"D+":5,\
              "F+":3,"A-":9,"B-":7,"C-":5,"D-":3,"F-":0, '3 1/2': 3.5, 'T': None, 'N':None, 'R':None}
    
    if rating in grades.keys():
        norm=grades[rating]
        return norm
    elif ('/') in rating:
        r = rating.split('/')
        norm = float(r[0])/float(r[1])*10
        return norm
    elif ('-') in rating and len(rating) > 2:
        r = rating.split('-')
        norm = float(r[0])/float(r[1])*10
        return norm
    else:
        norm = float(rating)
        return norm

    
    
rt_df.dropna(subset=['rating_y'],inplace=True)    
rt_df['critic_rating'] = rt_df['rating_y'].apply(norm_ratings)

In [306]:
# CLEAN RUNTIMES FLOAT>INT

def clean_runtime(runtime):
    minutes = runtime.split()[0]
    return int(minutes) 


rt_df.dropna(subset=['runtime'],inplace=True)    
rt_df['runtime'] = rt_df['runtime'].apply(clean_runtime)
rt_df.drop(columns = 'rating_y', inplace = True)
rt_df.head()

,id,director,runtime,rating_x,critic,critic_rating
0,3,David Cronenberg,108,R,PJ Nabarro,6.0
6,3,David Cronenberg,108,R,Eric D. Snider,6.0
7,3,David Cronenberg,108,R,Matt Kelemen,4.0
11,3,David Cronenberg,108,R,Emanuel Levy,7.0
12,3,David Cronenberg,108,R,Christian Toto,5.0


In [307]:
# ADDING DIRECTOR NAME ID TO RT_DF

name_key = file_dict['imdb.name.basics'][['nconst', 'primary_name']]




rt_df2 = rt_df.merge(name_key, left_on = 'director', right_on = 'primary_name')
rt_df2 = rt_df2.drop(columns = 'primary_name')

rt_df2.head()

,id,director,runtime,rating_x,critic,critic_rating,nconst
0,3,David Cronenberg,108,R,PJ Nabarro,6.0,nm0000343
1,3,David Cronenberg,108,R,Eric D. Snider,6.0,nm0000343
2,3,David Cronenberg,108,R,Matt Kelemen,4.0,nm0000343
3,3,David Cronenberg,108,R,Emanuel Levy,7.0,nm0000343
4,3,David Cronenberg,108,R,Christian Toto,5.0,nm0000343


In [308]:
movie_director_code = file_dict['imdb.title.crew'][['tconst', 'directors']]
movie_director_code['director'] = movie_director_code['directors'].str.split(',')
movie_director_code.drop(columns = 'directors', inplace = True)
movie_director_code = movie_director_code.explode('director')



In [309]:
movie_id_names_time = file_dict['imdb.title.basics'][['tconst', 'primary_title', 'original_title', 'runtime_minutes']]
half_way_clean = movie_id_names_time.merge(movie_director_code, how = 'inner', on = 'tconst')
half_way_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169262 entries, 0 to 169261
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           169262 non-null  object 
 1   primary_title    169262 non-null  object 
 2   original_title   169240 non-null  object 
 3   runtime_minutes  133324 non-null  float64
 4   director         163535 non-null  object 
dtypes: float64(1), object(4)
memory usage: 7.7+ MB


In [310]:
def float_conv(flt):
    integer = int(flt)
    return integer


half_way_clean.dropna(subset = ['runtime_minutes'], inplace = True)
half_way_clean['runtime_minutes'] = half_way_clean['runtime_minutes'].apply(float_conv)

In [311]:
# half_way_clean.head(1)
rt_df.head(1)

,id,director,runtime,rating_x,critic,critic_rating
0,3,David Cronenberg,108,R,PJ Nabarro,6.0


In [312]:
# combined_df = rt_df.merge(half_way_clean, left_on = ['runtime', 'nconst'], right_on = ['runtime_minutes', 'director'])

# len(combined_df['id'].unique())
combined_df.head()

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431


In [313]:
tester = combined_df.rename({'director_x': 'DirectorName', 'rating_x': "MPArating", 'nconst':'DirectorId'}, inplace = True)

combined_df
# tester.head()

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
...,...,...,...,...,...,...,...,...,...,...,...,...
6697,1944,Michel Gondry,119,PG-13,Edward Douglas,8.5,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6698,1944,Michel Gondry,119,PG-13,Matt Singer,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6699,1944,Michel Gondry,119,PG-13,Perri Nemiroff,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6700,1944,Michel Gondry,119,PG-13,Sam Bathe,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273


In [331]:
tester = combined_df.merge(file_dict['imdb.title.ratings'], on = 'tconst')
# tester2 = head()
# len(tester['id'].unique())
tester.info()
tester.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6686 entries, 0 to 6685
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               6686 non-null   int64  
 1   director_x       6686 non-null   object 
 2   runtime          6686 non-null   int64  
 3   rating_x         6686 non-null   object 
 4   critic           6494 non-null   object 
 5   critic_rating    6683 non-null   float64
 6   nconst           6686 non-null   object 
 7   tconst           6686 non-null   object 
 8   primary_title    6686 non-null   object 
 9   original_title   6686 non-null   object 
 10  runtime_minutes  6686 non-null   int64  
 11  director_y       6686 non-null   object 
 12  averagerating    6686 non-null   float64
 13  numvotes         6686 non-null   int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 783.5+ KB


,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y,averagerating,numvotes
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742
